In [1]:
import pandas as pd
import numpy as np

In [2]:
# importar pacotes usados na seleção do modelo e na medição da precisão
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

In [44]:
df = pd.read_csv('extra/nome_sexo_pf30.csv', index_col=0)
df = df.loc[~df.index.isna()]
df[df.index.isnull()]

,sexo
nome,


In [45]:
df.rename(columns={'sexo': 'SEXO'}, inplace=True)

In [46]:
df.head(10)

,SEXO
nome,
ABDEMES,M
ABILIS,M
ABNALDO,M
ADAILZA,M
ADALBERSON,M
ADALTINA,F
ADAMUS,M
ADEMILDA,F
ADEMISSON,M


In [47]:
df['SEXO'] = df['SEXO'].map({'F': 1, 'M': 2, 'X': 0}).astype('uint8')

In [48]:
df['REV'] = df.index
df['REV'] = df['REV'].apply(lambda x: x[::-1])

In [49]:
df.head()

,SEXO,REV
nome,,
ABDEMES,2,SEMEDBA
ABILIS,2,SILIBA
ABNALDO,2,ODLANBA
ADAILZA,2,AZLIADA
ADALBERSON,2,NOSREBLADA


In [50]:
MAIOR_QTDE_LETRAS = 16 # fixado para o maior arquivo

for i in range(MAIOR_QTDE_LETRAS):
    df['L' + str(i)] = df['REV'].apply(lambda x: ord(x[i]) - 64 if len(x) > i else 0).astype('uint8')

In [51]:
#df.dtypes

In [52]:
df.head(10)

,SEXO,REV,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
nome,,,,,,,,,,,,,,,,,,
ABDEMES,2,SEMEDBA,19,5,13,5,4,2,1,0,0,0,0,0,0,0,0,0
ABILIS,2,SILIBA,19,9,12,9,2,1,0,0,0,0,0,0,0,0,0,0
ABNALDO,2,ODLANBA,15,4,12,1,14,2,1,0,0,0,0,0,0,0,0,0
ADAILZA,2,AZLIADA,1,26,12,9,1,4,1,0,0,0,0,0,0,0,0,0
ADALBERSON,2,NOSREBLADA,14,15,19,18,5,2,12,1,4,1,0,0,0,0,0,0
ADALTINA,1,ANITLADA,1,14,9,20,12,1,4,1,0,0,0,0,0,0,0,0
ADAMUS,2,SUMADA,19,21,13,1,4,1,0,0,0,0,0,0,0,0,0,0
ADEMILDA,1,ADLIMEDA,1,4,12,9,13,5,4,1,0,0,0,0,0,0,0,0
ADEMISSON,2,NOSSIMEDA,14,15,19,19,9,13,5,4,1,0,0,0,0,0,0,0


In [53]:
#df.describe()

In [54]:
# definir dados de entrada

X = df.drop(['SEXO', 'REV'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo

print('Forma dos dados originais:', X.shape, y.shape)

Forma dos dados originais: (384846, 16) (384846,)


In [55]:
X.head()

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
nome,,,,,,,,,,,,,,,,
ABDEMES,19,5,13,5,4,2,1,0,0,0,0,0,0,0,0,0
ABILIS,19,9,12,9,2,1,0,0,0,0,0,0,0,0,0,0
ABNALDO,15,4,12,1,14,2,1,0,0,0,0,0,0,0,0,0
ADAILZA,1,26,12,9,1,4,1,0,0,0,0,0,0,0,0,0
ADALBERSON,14,15,19,18,5,2,12,1,4,1,0,0,0,0,0,0


In [56]:
y.head()

nome
ABDEMES       2
ABILIS        2
ABNALDO       2
ADAILZA       2
ADALBERSON    2
Name: SEXO, dtype: uint8

In [16]:
NUMBER_KFOLD_SPLITS = 5 # number of splits in cross-validation
NUMBER_GRID_ITERATIONS = 10 # number of grid iterations to parameters testing
SCORING_METRIC = 'accuracy' # the scoring metric to be used

In [17]:
from datetime import datetime

models = {}

def evaluate_model(name, model, X=X, y=y):

  start = datetime.now()
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)
  results = cross_val_score(model, X, y, cv=kfold,
                            scoring=SCORING_METRIC, verbose=1, n_jobs=-1)
  end = datetime.now()

  elapsed = int((end - start).total_seconds() * 1000)
  score = results.mean() * 100
  stddev = results.std() * 100

  models[name] = (model, score, stddev, elapsed)
  print(model, '\nCross-Validation Score: %.2f (+/- %.2f) [%5s ms]' % \
        (score, stddev, elapsed))
  return score, stddev, elapsed

In [18]:
# faz o ajuste fino do modelo, calculando os melhores hiperparâmetros
def fine_tune_model(model, params, X=X, y=y):

  print('\nFine Tuning Model:')
  print(model, "\nparams:", params)
  
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)

  search = RandomizedSearchCV(model, param_distributions=params, 
                              n_iter=NUMBER_GRID_ITERATIONS, 
                              scoring=SCORING_METRIC, cv=kfold, 
                              verbose=1, n_jobs=-1)
  #search = GridSearchCV(estimator=model, param_grid=params, scoring='accuracy', cv=kfold, verbose=1)

  search.fit(X, y)
  print('\nBest Score: %.2f %%' % (search.best_score_ * 100))
  print('Best Params:', search.best_params_)
  return search

In [19]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=42, multi_class='auto', max_iter=1000, solver='liblinear', C=0.01) # 85.18
evaluate_model('LR', model)

params = dict(solver=['liblinear','lbfgs'], C=np.logspace(-3,3,7))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LogisticRegression(C=0.01, max_iter=1000, random_state=42, solver='liblinear') 
Cross-Validation Score: 78.98 (+/- 0.30) [ 4803 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    4.7s finished


In [20]:
# Linear SVM
from sklearn.svm import LinearSVC

model = LinearSVC(random_state=42, max_iter=1000, C=0.001) # 85.27
evaluate_model('LSVM', model)

#from scipy.stats import reciprocal, uniform
#params = dict(C=uniform(1, 10))
params = dict(C=np.logspace(-4,3,8))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LinearSVC(C=0.001, random_state=42) 
Cross-Validation Score: 79.31 (+/- 0.32) [ 4020 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.9s finished


In [21]:
# K-Nearest Neighbours (KNN)
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=1) # 87.45
evaluate_model('KNN', model)

params = dict(n_neighbors=[1,3,5,7,9,11])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KNeighborsClassifier(n_neighbors=1) 
Cross-Validation Score: 76.49 (+/- 0.34) [16005 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   15.9s finished


In [22]:
# Linear Discriminant Analysis (LDA)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model = LinearDiscriminantAnalysis(solver='svd') # 84.45
evaluate_model('LDA', model)

params = dict(solver=['svd','lsqr','eigen'])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LinearDiscriminantAnalysis() 
Cross-Validation Score: 78.97 (+/- 0.29) [ 1454 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.3s finished


In [23]:
# Quadratic Discriminant Analysis (QDA)
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

model = QuadraticDiscriminantAnalysis() # 44.99
evaluate_model('QDA', model)

params = dict()
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


QuadraticDiscriminantAnalysis() 
Cross-Validation Score: 54.03 (+/- 0.16) [  738 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.6s finished


In [24]:
# Naïve Bayes - Gaussian
from sklearn.naive_bayes import GaussianNB

model = GaussianNB(priors=None, var_smoothing=0.1) # 67.62
evaluate_model('GNB', model)

params = dict(var_smoothing=np.logspace(0,-9,num=10))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


GaussianNB(var_smoothing=0.1) 
Cross-Validation Score: 78.99 (+/- 0.50) [  666 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s finished


In [25]:
# Naïve Bayes - Bernoulli
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB() # 43.37
evaluate_model('BNB', model)

params = dict(var_smoothing=np.logspace(0,-9,num=10))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


BernoulliNB() 
Cross-Validation Score: 54.45 (+/- 0.40) [  731 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.6s finished


In [26]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=17) # 96.37
evaluate_model('DT', model)

params = dict(criterion=['gini','entropy'], max_depth=[3,5,7,11,13,17,19])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


DecisionTreeClassifier(criterion='entropy', max_depth=17, random_state=42) 
Cross-Validation Score: 84.22 (+/- 0.31) [ 1777 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.7s finished


In [27]:
# Ensemble - Random Forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42, max_features='auto', n_estimators=500) # 93.86
evaluate_model('RF', model)

params = dict(n_estimators=[10,50,100,500], max_features=['auto','sqrt','log2'])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


RandomForestClassifier(n_estimators=500, random_state=42) 
Cross-Validation Score: 82.63 (+/- 0.31) [165608 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.8min finished


In [28]:
# Ensemble - Ada Boost
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=17), n_estimators=13) # 96.41
evaluate_model('ABDT', model)

params = dict(n_estimators=[1,3,5,7,9,11,13])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   18.5s finished


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                         max_depth=17,
                                                         random_state=42),
                   n_estimators=13) 
Cross-Validation Score: 80.72 (+/- 0.47) [18770 ms]


In [29]:
# Stochastic Gradient Descent (SGD)
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(random_state=42, max_iter=500, tol=0.1) # 77.02
evaluate_model('SGD', model)

params = dict(max_iter=[100, 200, 350, 500, 1000], tol=[0.01, 0.1, 1.0])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


SGDClassifier(max_iter=500, random_state=42, tol=0.1) 
Cross-Validation Score: 66.42 (+/- 7.79) [ 1807 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.7s finished


In [30]:
# Perceptron
from sklearn.linear_model import Perceptron

model = Perceptron(random_state=42, max_iter=500, tol=0.001) # 75.58
evaluate_model('PCT', model)

params = dict(max_iter=[100, 200, 350, 500, 750, 1000], tol=[0.1, 0.01, 0.001])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.9s finished


Perceptron(max_iter=500, random_state=42) 
Cross-Validation Score: 65.45 (+/- 7.32) [ 1144 ms]


In [31]:
# Multi-Layer Perceptron (MLP)
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(random_state=42, solver='lbfgs', alpha=1e-07, hidden_layer_sizes=(50, 100, 50)) # 88.67
evaluate_model('MLP', model)

params = dict(alpha=np.logspace(-9,-1,5), 
              solver=['lbfgs','sgd','adam'], 
              hidden_layer_sizes=[(50,50,50), (50,100,50), (100,)])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


MLPClassifier(alpha=1e-07, hidden_layer_sizes=(50, 100, 50), random_state=42,
              solver='lbfgs') 
Cross-Validation Score: 83.92 (+/- 0.27) [415783 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  6.9min finished


In [32]:
# Extreme Gradient Boosting Machine (XGBM)
from xgboost import XGBClassifier

model = XGBClassifier(max_depth=9, min_child_weight=1, gamma=0.1, subsample=0.9, colsample_bytree=0.7, reg_alpha=1e-05) # 97.67
evaluate_model('XGB', model)

params = dict(max_depth=range(3,10,2), min_child_weight=range(1,6,2)) # 97.60
#params = dict(gamma=[i/10.0 for i in range(0,5)]) # 97.68
#params = dict(subsample=[i/10.0 for i in range(6,10)], colsample_bytree=[i/10.0 for i in range(6,10)]) # 97.72
#params = dict(reg_alpha=[1e-5, 1e-2, 0.1, 1, 100]) # 97.74
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.7, gamma=0.1,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=1e-05, reg_lambda=None,
              scale_pos_weight=None, subsample=0.9, tree_method=None,
              validate_parameters=None, verbosity=None) 
Cross-Validation Score: 87.55 (+/- 0.26) [46591 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   46.5s finished


In [33]:
# LightGBM
from lightgbm import LGBMClassifier
#!pip install lightgbm

model = LGBMClassifier(num_leaves=73, max_depth=24, learning_rate=0.5, feature_fraction=0.75) # 97.51
evaluate_model('LGBM', model)

#params = dict(num_leaves=range(10,100), max_depth=range(1,30)) # 97.18
#params = dict(learning_rate=[0.01,0.05,0.1,0.5]) # 97.51
#params = dict(feature_fraction=[0.1,0.25,0.5,0.75,1.0]) # 97.62
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/home/rodrigo/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


LGBMClassifier(feature_fraction=0.75, learning_rate=0.5, max_depth=24,
               num_leaves=73) 
Cross-Validation Score: 87.32 (+/- 0.26) [10388 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   10.3s finished


(87.31808453718568, 0.2579340993727955, 10388)

In [34]:
# CatBoost
from catboost import CatBoostClassifier
#!pip install catboost

model = CatBoostClassifier(silent=True) # 97.04
#learning_rate=0.1, iterations=1000, depth=5
evaluate_model('CB', model)

params = dict(iterations=[10,50,100,250,500,1000],
              learning_rate=[0.01,0.05,0.1,0.5],
              depth=range(1,11,2)) 
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Cross-Validation Score: 87.98 (+/- 0.26) [130818 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.2min finished


In [35]:
names = []
estimators = []
scores = []
stdevs = []
times = []

for key, value in models.items():
  (model, score, stdev, elapsed) = value
  names.append(key)
  estimators.append(model)
  scores.append(score)
  stdevs.append(stdev)
  times.append(elapsed)

In [36]:
results_df = pd.DataFrame({
    'Model': names,
    'Score': scores,
    'Std Dev': stdevs,
    'Time (ms)': times,
    'Estimator': estimators})

results_df.sort_values(by='Score', ascending=False)

,Model,Score,Std Dev,Time (ms),Estimator
15,CB,87.977528,0.259532,130818,<catboost.core.CatBoostClassifier object at 0x...
13,XGB,87.554842,0.263357,46591,"XGBClassifier(base_score=None, booster=None, c..."
14,LGBM,87.318085,0.257934,10388,"LGBMClassifier(feature_fraction=0.75, learning..."
7,DT,84.216158,0.313470,1777,"DecisionTreeClassifier(criterion='entropy', ma..."
12,MLP,83.915195,0.273868,415783,"MLPClassifier(alpha=1e-07, hidden_layer_sizes=..."
8,RF,82.625736,0.310816,165608,"RandomForestClassifier(n_estimators=500, rando..."
9,ABDT,80.722311,0.471926,18770,AdaBoostClassifier(base_estimator=DecisionTree...
1,LSVM,79.311129,0.316158,4020,"LinearSVC(C=0.001, random_state=42)"
5,GNB,78.990102,0.503441,666,GaussianNB(var_smoothing=0.1)
0,LR,78.978063,0.299410,4803,"LogisticRegression(C=0.01, max_iter=1000, rand..."


In [37]:
model = models['CB'][0]
model

In [38]:
model.fit(X, y)

In [57]:
y_pred = model.predict(X)
y_pred

array([2, 2, 2, ..., 2, 1, 1])

In [58]:
dados = pd.DataFrame({'REAL': y, 'PREV': y_pred}, index=X.index)
for col in dados.columns:
    dados[col] = dados[col].map({0: 'X', 1: 'F', 2: 'M'})
dados.head()

,REAL,PREV
nome,,
ABDEMES,M,M
ABILIS,M,M
ABNALDO,M,M
ADAILZA,M,F
ADALBERSON,M,M


In [59]:
from sklearn.metrics import accuracy_score

accuracy_score(y, y_pred)

0.852234400253608

In [60]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y, y_pred)

array([[172931,  27944],
       [ 28923, 155048]])

In [61]:
dados[dados['REAL'] != dados['PREV']]

,REAL,PREV
nome,,
ADAILZA,M,F
ADEVIR,F,M
ALDOMAR,F,M
ALDRIAN,F,M
ALDRIM,F,M
...,...,...
WENCESLAU,F,M
YAQIN,F,M
ZENEDIR,M,F
